In [122]:
%matplotlib inline
import matplotlib.pyplot as plt 
import pandas 
import numpy as np 

In [123]:
dfs = []
n_pts = np.random.randint( 0, 100 )
for pt in range( n_pts ):
    n_samples = np.random.randint( 10, 384 )
    dat = [( 'sample_{0:04d}'.format(n), 'T7', 'T7term' ) for n in range( n_samples ) ]
    df = pandas.DataFrame(dat, columns=[ 'sample_name', 'fwd', 'rev' ]) 
    df[ 'source_pt' ] = pt
    dfs.append( df ) 
df = pandas.concat( dfs ) 
df.reset_index( inplace=True ) 

In [124]:
df.shape

(360, 5)

In [125]:
df.sample(5)

,index,sample_name,fwd,rev,source_pt
333,281,sample_0281,T7,T7term,1
346,294,sample_0294,T7,T7term,1
348,296,sample_0296,T7,T7term,1
124,72,sample_0072,T7,T7term,1
355,303,sample_0303,T7,T7term,1


In [126]:
df[ 'dest_pt' ] = df.index.map( lambda x: x // 1536 ) 
df[ 'dest_well' ] = df.index.map( lambda x: x % 1536 ) 
df.sample( 5 ) 

,index,sample_name,fwd,rev,source_pt,dest_pt,dest_well
139,87,sample_0087,T7,T7term,1,0,139
55,3,sample_0003,T7,T7term,1,0,55
287,235,sample_0235,T7,T7term,1,0,287
31,31,sample_0031,T7,T7term,0,0,31
188,136,sample_0136,T7,T7term,1,0,188


In [127]:
fmt_str = '''
This operation will condense {n_sample_pt} sample plates
into {n_pcr_pt} PCR plates using {n_ops:,} Echo operations
and {n_pt_ops} arm moves 
'''

params = {
    'n_sample_pt': len( df.source_pt.unique() ), 
    'n_pcr_pt': len( df.dest_pt.unique() ),
    'n_ops': len( df ) * 3, 
    'n_pt_ops': len( df.source_pt.unique() ) * 2, 
}

print( fmt_str.format( **params ) ) 


This operation will condense 2 sample plates
into 1 PCR plates using 1,080 Echo operations
and 4 arm moves 



In [132]:
arm_moves = 0
echo_moves = 0 
seal_moves = 0

for dest_pt, g in df.groupby( 'dest_pt' ):
    print( 'Arm: empty PCR plate to Echo' ) 
    arm_moves += 1
    for sample_pt, h in g.groupby( 'source_pt' ):
        print( 'Arm: sample plate {} to Echo'.format( sample_pt ) )
        arm_moves += 1 
        for idx, row in h.iterrows():
            #print( 'Echo: 5 nL from {} to {}'.format( row['index'], row['dest_well'] ) ) 
            echo_moves += 1
        print( 'Arm: sample plate {} to sealer'.format( sample_pt ) )
        arm_moves += 1
        print( 'Sealer: seal' ) 
        seal_moves += 1
        print( 'Arm: sample plate {} to storage'.format( sample_pt ) )
        arm_moves += 1
                    
for fwd_primer, g in df.groupby( 'fwd' ):
    print( 'Arm: primer plate {} to Echo'.format( fwd_primer ) ) 
    arm_moves += 1
    for idx, row in g.iterrows():
        echo_moves += 1 
    print( 'Arm: primer plate {} to sealer'.format( fwd_primer ) )
    arm_moves += 1
    print( 'Sealer: seal primer plate {}'.format( fwd_primer ) )
    seal_moves += 1 
    print( 'Arm: primer plate {} to hotel'.format( fwd_primer ) )
        
for fwd_primer, g in df.groupby( 'rev' ):
    print( 'Arm: primer plate {} to Echo'.format( fwd_primer ) ) 
    arm_moves += 1
    for idx, row in g.iterrows():
        echo_moves += 1 
    print( 'Arm: primer plate {} to sealer'.format( fwd_primer ) )
    arm_moves += 1
    print( 'Sealer: seal primer plate {}'.format( fwd_primer ) )
    seal_moves += 1 
    print( 'Arm: primer plate {} to hotel'.format( fwd_primer ) )

seconds_per_echo = 0.2 
seconds_per_arm = 15 
seconds_per_seal = 45
total_seconds = \
    ( seconds_per_arm * arm_moves ) + \
    ( seconds_per_echo * echo_moves ) + \
    ( seconds_per_seal * seal_moves ) 
m, s = divmod(total_seconds, 60)
h, m = divmod(m, 60)



fmt_str = '''
This operation will condense {n_sample_pt} sample plates
into {n_pcr_pt} PCR plate{plural} using {n_ops:,} Echo operations
and {n_pt_ops} arm moves 
'''

params = {
    'n_sample_pt': len( df.source_pt.unique() ), 
    'n_pcr_pt': len( df.dest_pt.unique() ),
    'n_ops': echo_moves, 
    'n_pt_ops': arm_moves, 
    'plural': 's',
}

print( fmt_str.format( **params ) ) 

print( 'Total time:', "%d hr %02d min %02d sec" % (h, m, s) )

Arm: empty PCR plate to Echo
Arm: sample plate 0 to Echo
Arm: sample plate 0 to sealer
Sealer: seal
Arm: sample plate 0 to storage
Arm: sample plate 1 to Echo
Arm: sample plate 1 to sealer
Sealer: seal
Arm: sample plate 1 to storage
Arm: primer plate T7 to Echo
Arm: primer plate T7 to sealer
Sealer: seal primer plate T7
Arm: primer plate T7 to hotel
Arm: primer plate T7term to Echo
Arm: primer plate T7term to sealer
Sealer: seal primer plate T7term
Arm: primer plate T7term to hotel

This operation will condense 2 sample plates
into 1 PCR plates using 1,080 Echo operations
and 11 arm moves 

Total time: 0 hr 09 min 21 sec
